Markdown cell 1 — Purpose
```markdown
Compute Tier A & Tier B facial geometry metrics using MediaPipe FaceMesh (CPU-only) over raw images in `data/raw/keep_local_125000_files/`, and write results to `data/processed/metadata/attributes_with_meta.parquet`.
- Metrics: symmetry, eye/face ratio, golden-ratio deviation, jaw width/chin height, smile intensity, and supporting proportions.
- Handles corrupted/missing images gracefully; skips failures.
- Uses CPU-safe FaceMesh (static_image_mode=True, max_num_faces=1, refine_landmarks=True).
```


In [14]:
import os
from pathlib import Path
import cv2
import numpy as np
import polars as pl
from tqdm import tqdm
import mediapipe as mp

# Resolve repo root (works when running from notebooks/)
ROOT = Path.cwd()
if ROOT.name == 'notebooks':
    ROOT = ROOT.parent

# Paths (V4 structure)
RAW_DIR = ROOT / "data/raw"
OUT_PATH = ROOT / "data/processed/metadata/attributes_with_meta.parquet"
OUT_PATH.parent.mkdir(parents=True, exist_ok=True)

In [15]:
# ```python
mp_face_mesh = mp.solutions.face_mesh

FACE_MESH_KWARGS = dict(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

# Key landmark indices (FaceMesh)
IDX_LEFT_EYE_OUTER = 33
IDX_LEFT_EYE_INNER = 133
IDX_RIGHT_EYE_INNER = 362
IDX_RIGHT_EYE_OUTER = 263
IDX_NOSE_TIP = 1
IDX_CHIN = 152
IDX_FOREHEAD = 10
IDX_MOUTH_LEFT = 61
IDX_MOUTH_RIGHT = 291
IDX_JAW_LEFT = 234
IDX_JAW_RIGHT = 454


def extract_landmarks(image_path, face_mesh):
    'Return np.array shape (N,3) of landmarks in image coords; None on failure.'
    try:
        img = cv2.imread(str(image_path))
        if img is None:
            return None
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    except Exception:
        return None

    h, w, _ = rgb.shape
    try:
        result = face_mesh.process(rgb)
    except Exception:
        return None

    if not result.multi_face_landmarks:
        return None

    lm = result.multi_face_landmarks[0].landmark
    pts = np.array([[p.x * w, p.y * h, p.z] for p in lm], dtype=np.float32)
    return pts


def _safe_dist(p1, p2):
    return float(np.linalg.norm(p1 - p2))


def compute_symmetry(pts):
    'Symmetry score: lower is more symmetric (mean abs x-diff to midline).'
    cx = float(np.mean(pts[:, 0]))
    x_diff = np.abs((pts[:, 0] - cx))
    return float(np.mean(x_diff) / (np.ptp(pts[:, 0]) + 1e-6))


def compute_proportions(pts):
    'Return eye/face ratio and golden-ratio deviation.'
    eye_width_left = _safe_dist(pts[IDX_LEFT_EYE_OUTER], pts[IDX_LEFT_EYE_INNER])
    eye_width_right = _safe_dist(pts[IDX_RIGHT_EYE_INNER], pts[IDX_RIGHT_EYE_OUTER])
    interpupil = _safe_dist((pts[IDX_LEFT_EYE_INNER] + pts[IDX_RIGHT_EYE_INNER]) / 2,
                            (pts[IDX_LEFT_EYE_OUTER] + pts[IDX_RIGHT_EYE_OUTER]) / 2)
    face_height = _safe_dist(pts[IDX_FOREHEAD], pts[IDX_CHIN])
    face_width = _safe_dist(pts[IDX_JAW_LEFT], pts[IDX_JAW_RIGHT])
    eye_face_ratio = (eye_width_left + eye_width_right + interpupil) / 3.0 / (face_width + 1e-6)
    golden_target = 1.618
    golden_ratio = (face_height / (face_width + 1e-6))
    golden_deviation = abs(golden_ratio - golden_target)
    return eye_face_ratio, golden_ratio, golden_deviation, face_height, face_width


def compute_jaw_metrics(pts, face_height):
    jaw_width = _safe_dist(pts[IDX_JAW_LEFT], pts[IDX_JAW_RIGHT])
    chin_height = _safe_dist(pts[IDX_NOSE_TIP], pts[IDX_CHIN])
    jaw_ratio = jaw_width / (face_height + 1e-6)
    chin_ratio = chin_height / (face_height + 1e-6)
    return jaw_width, chin_height, jaw_ratio, chin_ratio


def compute_smile_intensity(pts, face_width):
    mouth_width = _safe_dist(pts[IDX_MOUTH_LEFT], pts[IDX_MOUTH_RIGHT])
    smile_intensity = mouth_width / (face_width + 1e-6)
    return mouth_width, smile_intensity
# ```

In [16]:
from math import ceil

BATCH_SIZE = 512


def process_batch(paths, face_mesh):
    rows = []
    for path in paths:
        pts = extract_landmarks(path, face_mesh)
        if pts is None:
            continue
        sym = compute_symmetry(pts)
        eye_face_ratio, golden_ratio, golden_dev, face_h, face_w = compute_proportions(pts)
        jaw_w, chin_h, jaw_ratio, chin_ratio = compute_jaw_metrics(pts, face_h)
        mouth_w, smile_intensity = compute_smile_intensity(pts, face_w)

        rows.append({
            'filename': path.name,
            'symmetry_score': sym,
            'eye_face_ratio': eye_face_ratio,
            'golden_ratio': golden_ratio,
            'golden_deviation': golden_dev,
            'face_height': face_h,
            'face_width': face_w,
            'jaw_width': jaw_w,
            'chin_height': chin_h,
            'jaw_ratio': jaw_ratio,
            'chin_ratio': chin_ratio,
            'mouth_width': mouth_w,
            'smile_intensity': smile_intensity,
        })
    return rows


all_images = sorted([p for p in RAW_DIR.glob('*.jpg')])
if not all_images:
    raise SystemExit(f'No images found in {RAW_DIR}')

all_rows = []
with mp_face_mesh.FaceMesh(**FACE_MESH_KWARGS) as fm:
    total_batches = ceil(len(all_images) / BATCH_SIZE)
    for i in tqdm(range(total_batches), desc='FaceMesh batches'):
        batch = all_images[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        all_rows.extend(process_batch(batch, fm))

if not all_rows:
    raise SystemExit('No rows produced; check inputs and FaceMesh configuration')

df = pl.DataFrame(all_rows)
df.write_parquet(OUT_PATH)
print(f'Wrote {len(df)} rows → {OUT_PATH}')


I0000 00:00:1763934097.573613 3115269 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1
FaceMesh batches:   0%|          | 0/2 [00:00<?, ?it/s]W0000 00:00:1763934097.578088 3140073 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763934097.598017 3140071 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
FaceMesh batches: 100%|██████████| 2/2 [00:06<00:00,  3.40s/it]

Wrote 700 rows → /Users/jayklarin/__DI/Repositories/FaceStats/data/processed/metadata/attributes_with_meta.parquet


In [17]:
# ```python
if OUT_PATH.exists():
    preview = pl.read_parquet(OUT_PATH, n_rows=5)
    print(preview)
else:
    print(f'Missing output: {OUT_PATH}')
# ```

shape: (5, 13)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ filename  ┆ symmetry_ ┆ eye_face_ ┆ golden_ra ┆ … ┆ jaw_ratio ┆ chin_rati ┆ mouth_wid ┆ smile_in │
│ ---       ┆ score     ┆ ratio     ┆ tio       ┆   ┆ ---       ┆ o         ┆ th        ┆ tensity  │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ f64       ┆ ---       ┆ ---       ┆ ---      │
│           ┆ f64       ┆ f64       ┆ f64       ┆   ┆           ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ SFHQ_pt4_ ┆ 0.190882  ┆ 0.144021  ┆ 1.223847  ┆ … ┆ 0.817096  ┆ 0.40403   ┆ 185.29066 ┆ 0.349405 │
│ 00000019. ┆           ┆           ┆           ┆   ┆           ┆           ┆ 5         ┆          │
│ jpg       ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ SFHQ_pt4_ ┆ 0.191912  ┆ 0.134797  ┆ 1.184565  ┆ … ┆ 0.844192  ┆ 0.4566    

Markdown cell 2 — Geometry-based attractiveness score
```markdown
Compute a simple 1–10 attractiveness bucket from FaceMesh geometry metrics (symmetry, proportions, jaw/chin, smile). Lower symmetry_score is better; other metrics are scaled via z-scores. We rank by composite score and bin into deciles.
```


In [18]:
# ```python
import numpy as np
import pandas as pd

pdf = pl.read_parquet(OUT_PATH).to_pandas()

metrics = [
    "symmetry_score",        # lower is better
    "eye_face_ratio",
    "golden_ratio",         # closer to ~1.618 is better
    "jaw_ratio",
    "chin_ratio",
    "smile_intensity",
]

if pdf.empty:
    raise SystemExit("No rows to score; run FaceMesh extraction first.")

# z-score helper
scores = []
for col in metrics:
    vals = pdf[col].astype(float)
    mean = vals.mean()
    std = vals.std() or 1e-6
    z = (vals - mean) / std
    if col == "symmetry_score":
        z = -z  # lower symmetry_score is better
    if col == "golden_ratio":
        target = 1.618
        z = -np.abs((vals - target) / std)  # penalize deviation from golden ratio
    scores.append(z)

pdf["geom_score"] = np.vstack(scores).mean(axis=0)

# Percentile-based 1–10 buckets
percentile = pdf["geom_score"].rank(pct=True)
pdf["attractiveness_geom"] = np.clip(np.ceil(percentile * 10), 1, 10).astype(int)

# Write back to parquet
pl.DataFrame(pdf).write_parquet(OUT_PATH)

print("Saved with attractiveness_geom column →", OUT_PATH)
print("Distribution (1–10):")
print(pdf["attractiveness_geom"].value_counts().sort_index())
# ```

Saved with attractiveness_geom column → /Users/jayklarin/__DI/Repositories/FaceStats/data/processed/metadata/attributes_with_meta.parquet
Distribution (1–10):
attractiveness_geom
1     70
2     70
3     70
4     70
5     70
6     70
7     70
8     70
9     70
10    70
Name: count, dtype: int64
